In [2]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_xy,check_doi_glutton,normalize,aplatir,remove_too_long
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [3]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [4]:
pd.Series([list(pd.Series([x[i].get('wg') for i in range(len(x))]).drop_duplicates()) for x in list(df_ipcc.drop_duplicates(subset=['doi']).dropna(subset=['doi']).wg_chap)]).value_counts()

[2]                   21803
[3]                   12324
[1]                   10478
[2_cross]              3491
[2, 2_cross]           1531
[1, 2]                 1459
[2, 3]                 1226
[1, 2, 2_cross]         251
[1, 3]                  189
[1, 2_cross]            176
[1, 2, 3]               172
[2, 2_cross, 3]          94
[2_cross, 3]             34
[1, 2, 2_cross, 3]       24
[1, 2_cross, 3]           6
Name: count, dtype: int64

In [5]:
min([int(x) for x in list(df_ipcc.year_OA.drop_duplicates().dropna())])

1686

In [6]:
df_ipcc.test_glutton.value_counts()

test_glutton
True    74098
Name: count, dtype: int64

In [7]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [8]:
df_ipcc.columns

Index(['year', 'url', 'doi', 'title', 'author', 'freq', 'wg', 'chap',
       'countries', 'concepts', 'sdg', 'topics', 'is_OA_available', 'title_OA',
       'year_OA', 'authors_name', 'rors', 'institutions_names',
       'locations_names', 'ipcc', 'test_glutton'],
      dtype='object')

In [9]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [10]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [11]:
len(df_ipcc_group.doi.drop_duplicates())==len(df_ipcc_group.doi)

True

In [12]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [13]:
df_ipbes.columns

Index(['doi', 'title', 'date', 'creators', 'tags', 'countries', 'concepts',
       'sdg', 'topics', 'is_OA_available', 'year_OA', 'authors_name', 'rors',
       'institutions_names', 'locations_names', 'year'],
      dtype='object')

In [14]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [15]:
df_ipbes.countries.value_counts()

countries
[US]                        1164
[None]                       932
[GB]                         338
[AU]                         223
[CA]                         214
                            ... 
[BR, IT, HK]                   1
[US, SE, JP]                   1
[MY, CH, AU, US, FI, GB]       1
[EC, BR, ID, JP]               1
[US, CA, CL]                   1
Name: count, Length: 1573, dtype: int64

In [16]:
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [18]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0]})
df_ipbes_group=df_ipbes_group.reset_index()

In [19]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [20]:
len(df_ipbes_group.doi.drop_duplicates())==len(df_ipbes_group.doi)

True

In [21]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')

In [23]:
df_ipcc_ipbes['countries']=df_ipcc_ipbes.apply(lambda row: get_xy('countries',row), axis=1)
df_ipcc_ipbes['year']=df_ipcc_ipbes.apply(lambda row: get_xy('year',row), axis=1)
df_ipcc_ipbes['title']=df_ipcc_ipbes.apply(lambda row: get_xy('title',row), axis=1)
df_ipcc_ipbes['rors']=df_ipcc_ipbes.apply(lambda row: get_xy('rors',row), axis=1)
df_ipcc_ipbes['authors_name']=df_ipcc_ipbes.apply(lambda row: get_xy('authors_name',row), axis=1)
df_ipcc_ipbes['institutions_names']=df_ipcc_ipbes.apply(lambda row: get_xy('institutions_names',row), axis=1)

In [25]:
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year','title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes']]

In [27]:
len(df_ipcc_ipbes.doi.drop_duplicates())==len(df_ipcc_ipbes.doi)

True

In [77]:
update_bso_publications()

2024-04-04 11:40:55,440 | code_utils.BSO | DEBUG | update bso publications files


DEBUG:code_utils.BSO:update bso publications files
c:\Users\haallat\Documents\teds\code_utils\BSO.py:34: DtypeWarning: Columns (17,18,20,21,22,23,24,25,27,28,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bso = pd.read_csv('https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-publications-latest.csv.gz', sep=';')


2024-04-04 11:44:53,799 | code_utils.BSO | DEBUG | writing 1814868 dois info from bso publications


DEBUG:code_utils.BSO:writing 1814868 dois info from bso publications


In [30]:
bso=pd.DataFrame(get_bso_publications())

In [31]:
df_bso=bso.transpose().reset_index()

In [32]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [33]:
df_bso

,doi,rors,bso_local_affiliations
0,10.1007/s00442-013-2799-0,[05kpkpg04],[331596270]
1,10.1007/s10682-014-9701-3,[05kpkpg04],[331596270]
2,10.9734/bmrj/2014/4284,[05kpkpg04],[331596270]
3,10.1007/s10886-014-0480-7,[05kpkpg04],[331596270]
4,10.18278/wfp.2.1.5,[05kpkpg04],[331596270]
...,...,...,...
1814863,10.1101/147884,[],[]
1814864,10.1111/oik.05851,"[051escj72, 013cjyk83, 046b3cj80, 02rx3b187, 0...","[130029796, 130026149, 197534860, 130026081, 1..."
1814865,10.1101/2020.04.15.044065,[],[]
1814866,10.1101/2020.05.20.104802,[02en5vm52],"[130023385, 180089013]"


In [34]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [35]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [36]:
del df_ipcc_ipbes_bso['rors_y']

In [37]:
df_ipcc_ipbes_bso.columns

Index(['doi', 'year', 'title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes', 'bso_local_affiliations'],
      dtype='object')

In [38]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [39]:
df_iso=df_iso.transpose().reset_index().rename(columns={'index':'code',0:'name'})

In [40]:
dict_iso={}
for i,row in df_iso.iterrows():
    dict_iso[row['code']]=row['name']
dict_iso['CW']='Curaçao'
dict_iso['RS']='Serbia'
dict_iso['ME']='Montenegro'
dict_iso['XK']='Kosovo'
dict_iso['SS']='South Soudan'
dict_iso['MF']='Saint Martin'

In [41]:
df_ipcc_ipbes_bso.loc[:,'countries']=df_ipcc_ipbes_bso.loc[:,'countries'].apply(lambda x: [dict_iso[y] for y in x] if (x!=None)&(x!=[])&(isinstance(x,float)==False) else None)

In [42]:
df_brevet_doi=pd.read_json('brevet_doi.json')

FileNotFoundError: File brevet_doi.json does not exist

In [43]:
df_brevet_doi

NameError: name 'df_brevet_doi' is not defined

In [ ]:
df_ipcc_ipbes_bso_brevet=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [ ]:
len(df_ipcc_ipbes_bso_brevet.doi.drop_duplicates())==len(df_ipcc_ipbes_bso_brevet.doi)

In [ ]:
df_ipcc_ipbes_bso_brevet.doi.value_counts()[:10]

In [44]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

54821 data in cached openalex data
6607 data in cached openalex data


In [45]:
df_cache

,doi,OpenAlex_data
0,10.1098/rstl.1686.0026,"[{'id': 'https://openalex.org/W4244279015', 'd..."
1,10.1098/rstl.1735.0014,"[{'id': 'https://openalex.org/W4252425265', 'd..."
2,10.1098/rstl.1861.0001,"[{'id': 'https://openalex.org/W3091998362', 'd..."
3,10.1017/s0080456800017592,"[{'id': 'https://openalex.org/W2314134964', 'd..."
4,10.2307/2992177,"[{'id': 'https://openalex.org/W256209766', 'do..."
...,...,...
60244,10.9758/cpn.2015.13.3.239,"[{'id': 'https://openalex.org/W2221413160', 'd..."
60245,10.9774/gleaf.978-1-907643-06-4_5,"[{'id': 'https://openalex.org/W3124428460', 'd..."
60246,10.9783/9780812295948-002,"[{'id': 'https://openalex.org/W3137011924', 'd..."
60247,10.2172/2283221,"[{'id': 'https://openalex.org/W4391458189', 'd..."


In [46]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [47]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [48]:
dict_doi_bso=[]

In [49]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [50]:
len(dict_doi_bso)

58240

In [51]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [52]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [53]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', lines=True)

In [54]:
len(dd)

58240

In [55]:
current_dict = dd.to_dict(orient='records')

In [56]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [57]:
df_elastic_input=pd.DataFrame(current_dict)

In [58]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [59]:
df_elastic_input=df_elastic_input[['doi', 'year', 'title', 'rors', 'ipcc','authors_name', 'institutions_names',
       'countries', 'ipbes', 'openalex_authorships','openalex_topics']]

In [60]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [82]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [63]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [64]:
df.loc[:,'rors']=df.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'authors_name']=df.loc[:,'authors_name'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'institutions_names']=df.loc[:,'institutions_names'].apply(lambda x: [[]] if x==[None] else x)

In [65]:
df.loc[pd.isna(df['rors'])==False,'rors']=df.loc[pd.isna(df['rors'])==False,'rors'].apply(lambda x: ','.join([j.replace('https://ror.org/','') for j in list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())]))

In [66]:
df.loc[pd.isna(df['institutions_names'])==False,'institutions_names']=df.loc[pd.isna(df['institutions_names'])==False,'institutions_names'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())))

In [67]:
df.loc[pd.isna(df['authors_name'])==False,'authors_name']=df.loc[pd.isna(df['authors_name'])==False,'authors_name'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y.keys()) for y in x if isinstance(y,dict)])).drop_duplicates())))

In [70]:
df.loc[pd.isna(df['ipcc'])==False,'ipcc']=df.loc[pd.isna(df['ipcc'])==False,'ipcc'].apply(lambda x: ','.join(list((pd.Series([y.get('name') for y in x]).drop_duplicates()))))

In [71]:
df.loc[pd.isna(df['ipbes'])==False,'ipbes']=df.loc[pd.isna(df['ipbes'])==False,'ipbes'].apply(lambda x: ','.join(list((pd.Series([y.get('chapter') for y in x]).drop_duplicates()))))

In [ ]:
df['is_fr']=df['countries'].apply(lambda x: 'France' in x if isinstance(x,list) else False)

In [83]:
df.loc[pd.isna(df['countries'])==False,'countries']=df.loc[pd.isna(df['countries'])==False,'countries'].apply(lambda x: ','.join(x))

In [85]:
df[['doi','title','ipcc','ipbes','countries','is_fr','rors','institutions_names','authors_name']].to_excel('doi_ipcc_ipbes.xlsx', index=False)